In [4]:
import sympy as sp
import itertools
import pypolsys
import phcpy

In [5]:
class Game:
    def __init__(self, num_players, num_strategies, payoffs):
        self.num_players = num_players
        self.num_strategies = num_strategies
        self.payoffs = payoffs
        #self.indices_combinations = indices_combinations


def create_n_player_game(payoffs):
    num_players = len(payoffs)
    num_strategies = [len(payoffs[i]) for i in range(num_players)]
    #indices_combinations = list(itertools.product(*[range(num_players)] * len(num_strategies)))

    return Game(num_players, num_strategies, payoffs)

In [6]:
payoffs = [
    [
        [
            [1, 2],
            [3, 4]
        ],
        [
            [5, 6],
            [7, 8]
        ]
    ],
    [
        [
            [9, 10],
            [11, 12]
        ],
        [
            [13, 14],
            [15, 16]
        ]
    ],
    [
        [
            [17, 18],
            [19, 20]
        ],
        [
            [21, 22],
            [23, 24]
        ]
    ]
]

game = create_n_player_game(payoffs)
game.num_players, game.num_strategies, game.payoffs

(3,
 [2, 2, 2],
 [[[[1, 2], [3, 4]], [[5, 6], [7, 8]]],
  [[[9, 10], [11, 12]], [[13, 14], [15, 16]]],
  [[[17, 18], [19, 20]], [[21, 22], [23, 24]]]])

In [7]:
payoffs2 = [
    [
        [
            1,
            0
        ],
        [
            0,
            1
        ]
    ],
    [
        [
            1,
            0
        ],
        [
            0,
            1
        ]
    ]
]

game2 = create_n_player_game(payoffs2)
game2.num_players, game2.num_strategies, game2.payoffs

(2, [2, 2], [[[1, 0], [0, 1]], [[1, 0], [0, 1]]])

In [43]:
def calculate_expected_payoff_diff_equations(game):
    n = game.num_players
    # Create symbolic probability variables
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    def expected_payoff(player, strategy):
        s = game.num_strategies[player]
        payoff = 0

        # Create a modified list of strategy ranges, replacing the player's strategy range with [strategy]
        strategy_ranges = [range(game.num_strategies[p]) if p != player else [strategy] for p in range(n)]
        for indices in itertools.product(*strategy_ranges):
            probability_combination = [probabilities[p][indices[p]] if p != player else 1 for p in range(n)]
            payoff_value = game.payoffs[player][strategy]
            for i, index in enumerate(indices):
                if i != player:
                    payoff_value = payoff_value[index]
            payoff_term = payoff_value * sp.prod(probability_combination)

            payoff += payoff_term
        return payoff

#    def expected_payoff_from_mixed_strategy(player):
#        s = game.num_strategies[player]
#        total_payoff = 0
#        for strategy in range(s):
#            total_payoff += probabilities[player][strategy] * expected_payoff(player, strategy)
#        return total_payoff

    payoff_differences_equations = []
    for player in range(n):
        player_payoff_diff = []
        for strategy in range(game.num_strategies[player]-1):
            diff = expected_payoff(player, strategy) - expected_payoff(player, game.num_strategies[player]-1)
            player_payoff_diff.append(diff)
        payoff_differences_equations.append(player_payoff_diff)

    # Flatten the list of lists
    flat_payoff_differences_equations = [diff for sublist in payoff_differences_equations for diff in sublist]

    return flat_payoff_differences_equations

In [44]:
def calculate_expected_payoff_diff_equations_fj(game):
    n = game.num_players
    # Create symbolic probability variables
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    def expected_payoff(player, strategy):
        s = game.num_strategies[player]
        payoff = 0

        # Create a modified list of strategy ranges, replacing the player's strategy range with [strategy]
        strategy_ranges = [range(game.num_strategies[p]) if p != player else [strategy] for p in range(n)]
        for indices in itertools.product(*strategy_ranges):
            probability_combination = [probabilities[p][indices[p]] if p != player else 1 for p in range(n)]
            payoff_value = game.payoffs[player][strategy]
            for i, index in enumerate(indices):
                if i != player:
                    payoff_value = payoff_value[index]
            payoff_term = payoff_value * sp.prod(probability_combination)

            payoff += payoff_term
        return payoff

    def expected_payoff_from_mixed_strategy(player):
        s = game.num_strategies[player]
        total_payoff = 0
        for strategy in range(s):
            total_payoff += probabilities[player][strategy] * expected_payoff(player, strategy)
        return total_payoff

    payoff_differences_equations = []
    for player in range(n):
        player_payoff_diff = []
        for strategy in range(game.num_strategies[player]):
            diff = expected_payoff(player, strategy) - expected_payoff_from_mixed_strategy(player)
            player_payoff_diff.append(diff)
        payoff_differences_equations.append(player_payoff_diff)

    # Flatten the list of lists
    flat_payoff_differences_equations = [diff for sublist in payoff_differences_equations for diff in sublist]

    return flat_payoff_differences_equations

In [45]:
def calculate_jacobian(game):
    # Get the flattened payoff difference equations
    payoff_differences_equations = calculate_expected_payoff_diff_equations_fj(game)
    
    n = game.num_players
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    # Flatten the list of probability variables
    flat_probabilities = [p for sublist in probabilities for p in sublist]

    # Calculate the Jacobian
    jacobian_matrix = sp.Matrix(payoff_differences_equations).jacobian(flat_probabilities)

    return jacobian_matrix

In [10]:
def calculate_payoff_diff_and_prob_sum_equations(game):
    n = game.num_players
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    # Collect the equations in a list
    equations = []
    for player_payoff_diff in calculate_expected_payoff_diff_equations(game):
        equations.append(sp.poly(player_payoff_diff , *[p for player_probs in probabilities for p in player_probs]))

    for player_probs in probabilities:
        prob_sum_eq = sum(player_probs) - 1
        equations.append(sp.poly(prob_sum_eq, *[p for player_probs in probabilities for p in player_probs]))


    return equations

In [46]:
calculate_payoff_diff_and_prob_sum_equations(game2), calculate_jacobian(game2)

(['p21 - p22;', 'p11 - p12;', 'p11 + p12 - 1;', 'p21 + p22 - 1;'],
 Matrix([
 [   -p21,    -p22, 1 - p11,    -p12],
 [   -p21,    -p22,    -p11, 1 - p12],
 [1 - p21,    -p22,    -p11,    -p12],
 [   -p21, 1 - p22,    -p11,    -p12]]))

In [10]:
calculate_expected_payoff_diff_equations(game), calculate_jacobian(game)

([-p11*(p21*p31 + 2*p21*p32 + 3*p22*p31 + 4*p22*p32) - p12*(5*p21*p31 + 6*p21*p32 + 7*p22*p31 + 8*p22*p32) + p21*p31 + 2*p21*p32 + 3*p22*p31 + 4*p22*p32,
  -p11*(p21*p31 + 2*p21*p32 + 3*p22*p31 + 4*p22*p32) - p12*(5*p21*p31 + 6*p21*p32 + 7*p22*p31 + 8*p22*p32) + 5*p21*p31 + 6*p21*p32 + 7*p22*p31 + 8*p22*p32,
  9*p11*p31 + 10*p11*p32 + 11*p12*p31 + 12*p12*p32 - p21*(9*p11*p31 + 10*p11*p32 + 11*p12*p31 + 12*p12*p32) - p22*(13*p11*p31 + 14*p11*p32 + 15*p12*p31 + 16*p12*p32),
  13*p11*p31 + 14*p11*p32 + 15*p12*p31 + 16*p12*p32 - p21*(9*p11*p31 + 10*p11*p32 + 11*p12*p31 + 12*p12*p32) - p22*(13*p11*p31 + 14*p11*p32 + 15*p12*p31 + 16*p12*p32),
  17*p11*p21 + 18*p11*p22 + 19*p12*p21 + 20*p12*p22 - p31*(17*p11*p21 + 18*p11*p22 + 19*p12*p21 + 20*p12*p22) - p32*(21*p11*p21 + 22*p11*p22 + 23*p12*p21 + 24*p12*p22),
  21*p11*p21 + 22*p11*p22 + 23*p12*p21 + 24*p12*p22 - p31*(17*p11*p21 + 18*p11*p22 + 19*p12*p21 + 20*p12*p22) - p32*(21*p11*p21 + 22*p11*p22 + 23*p12*p21 + 24*p12*p22)],
 Matrix([
 [    

In [56]:
pol = pypolsys.utils.fromSympy(calculate_payoff_diff_and_prob_sum_equations(game2))
pypolsys.polsys.init_poly(*pol)
# Create homogeneous partition
part = pypolsys.utils.make_h_part(4)
# Pass it to POLSYS_PLP
pypolsys.polsys.init_partition(*part)
# Solve
bplp = pypolsys.polsys.solve(1e-8, 1e-15, 0.0)
# Get the roots, array of size (N+1) x bplp
r2 = pypolsys.polsys.myroots
# Get status of the solving process
pypolsys.polsys.report()

0

 Solve report :
 --------------
   Normal return.
   All paths were normally tracked.


In [57]:
r2

array([[0.5       +1.39770306e-13j],
       [0.5       -2.06213900e-13j],
       [0.5       +2.33466744e-13j],
       [0.5       -5.08511819e-13j],
       [0.23653383-5.41951312e-01j]])

In [47]:
s=phcpy.solve(calculate_payoff_diff_and_prob_sum_equations(game2))
s[1]

AttributeError: module 'phcpy' has no attribute 'solve'

In [8]:
x, y, z = sp.symbols('x, y, z')
sp.poly(x**2 + y + z - 1, (x, y, z))

Poly(x**2 + y + z - 1, x, y, z, domain='ZZ')

In [53]:
calculate_payoff_diff_and_prob_sum_equations(game2)

[Poly(p21 - p22, p11, p12, p21, p22, domain='ZZ'),
 Poly(p11 - p12, p11, p12, p21, p22, domain='ZZ'),
 Poly(p11 + p12 - 1, p11, p12, p21, p22, domain='ZZ'),
 Poly(p21 + p22 - 1, p11, p12, p21, p22, domain='ZZ')]

In [52]:
def calculate_payoff_diff_and_prob_sum_equations(game):
    n = game.num_players
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    # Collect the equations in a list
    equations = []
    for player_payoff_diff in calculate_expected_payoff_diff_equations(game):
        equations.append(sp.poly(player_payoff_diff , *[p for player_probs in probabilities for p in player_probs]))

    for player_probs in probabilities:
        prob_sum_eq = sum(player_probs) - 1
        equations.append(sp.poly(prob_sum_eq, *[p for player_probs in probabilities for p in player_probs]))


    return equations

In [49]:
def calculate_payoff_diff_and_prob_sum_equations(game):
    n = game.num_players
    probabilities = [[sp.Symbol(f'p{i+1}{j+1}') for j in range(game.num_strategies[i])] for i in range(n)]

    # Rest of the code remains the same

    # Collect the equations in a list
    equations = []
    for player_payoff_diff in calculate_expected_payoff_diff_equations(game):
        for eq in player_payoff_diff:
            equations.append(sp.poly(eq, *[p for player_probs in probabilities for p in player_probs]))

    for player_probs in probabilities:
        prob_sum_eq = sum(player_probs) - 1
        equations.append(sp.poly(prob_sum_eq, *[p for player_probs in probabilities for p in player_probs]))

    return equations

In [19]:
calculate_payoff_diff_and_prob_sum_equations(game2)

['-p11*p21 - p12*p22 + p21;',
 '-p11*p21 - p12*p22 + p22;',
 '-p11*p21 + p11 - p12*p22;',
 '-p11*p21 - p12*p22 + p12;',
 'p11 + p12 - 1;',
 'p21 + p22 - 1;']